In [3414]:
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from IPython.display import display
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [3455]:
x = np.random.rand(100, 2)
y = np.random.randint(0, 2, 100)

from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
base_estimator_bnb = BernoulliNB()

ada = AdaBoostClassifier(n_estimators=300, random_state=0)
ada.fit(x, y)
y_pred = ada.predict(x)
accuracy = np.sum(y_pred == y) / len(y)
print("Accuracy: ", accuracy)

Accuracy:  1.0


In [3416]:
df = pd.DataFrame(x, columns=['x1', 'x2'])
df = df.assign(y=y)

In [3417]:
df

,x1,x2,y
0,0.151174,0.354630,1
1,0.415353,0.984219,0
2,0.791493,0.903714,0
3,0.223697,0.077215,1
4,0.426906,0.121843,1
...,...,...,...
95,0.358278,0.046354,0
96,0.263334,0.842166,0
97,0.521146,0.931668,1
98,0.655226,0.428205,1


In [3418]:
knn1 = DecisionTreeClassifier(max_depth=1)
knn1.fit(df[['x1', 'x2']], df['y'])
y_pred = knn1.predict(df[['x1', 'x2']])

e = df['y'] != y_pred
e = e.astype(int)
df = df.assign(y_pred=y_pred)
df = df.assign(e=e)
df

,x1,x2,y,y_pred,e
0,0.151174,0.354630,1,1,0
1,0.415353,0.984219,0,0,0
2,0.791493,0.903714,0,0,0
3,0.223697,0.077215,1,1,0
4,0.426906,0.121843,1,1,0
...,...,...,...,...,...
95,0.358278,0.046354,0,1,1
96,0.263334,0.842166,0,0,0
97,0.521146,0.931668,1,0,1
98,0.655226,0.428205,1,1,0


In [3419]:
accuracy = np.sum(y_pred == df['y']) / len(df)
print("Accuracy: ", accuracy)

Accuracy:  0.58


In [3420]:
sw = 1/len(df)
df = df.assign(sample_weight=sw)
df

,x1,x2,y,y_pred,e,sample_weight
0,0.151174,0.354630,1,1,0,0.01
1,0.415353,0.984219,0,0,0,0.01
2,0.791493,0.903714,0,0,0,0.01
3,0.223697,0.077215,1,1,0,0.01
4,0.426906,0.121843,1,1,0,0.01
...,...,...,...,...,...,...
95,0.358278,0.046354,0,1,1,0.01
96,0.263334,0.842166,0,0,0,0.01
97,0.521146,0.931668,1,0,1,0.01
98,0.655226,0.428205,1,1,0,0.01


In [3421]:
error_rate = sum(df['sample_weight'] * (df['y'] != df['y_pred']))
lamda_rate = round(1/2 * np.log((1-error_rate)/error_rate),2)
alpha_1 = lamda_rate
display(df)
print('error_rate: ', error_rate)
print('lamda_rate: ', lamda_rate)

,x1,x2,y,y_pred,e,sample_weight
0,0.151174,0.354630,1,1,0,0.01
1,0.415353,0.984219,0,0,0,0.01
2,0.791493,0.903714,0,0,0,0.01
3,0.223697,0.077215,1,1,0,0.01
4,0.426906,0.121843,1,1,0,0.01
...,...,...,...,...,...,...
95,0.358278,0.046354,0,1,1,0.01
96,0.263334,0.842166,0,0,0,0.01
97,0.521146,0.931668,1,0,1,0.01
98,0.655226,0.428205,1,1,0,0.01


error_rate:  0.4200000000000002
lamda_rate:  0.16


In [3422]:
update_sample_weight = []
for i in range(len(df)):
    if df['e'][i] == 1:
        update_sample_weight.append(round(df['sample_weight'][i] * np.exp(lamda_rate), 2))
    else:
        update_sample_weight.append(round(df['sample_weight'][i] * np.exp(-lamda_rate), 2))
        
df = df.assign(update_sample_weight=update_sample_weight)
df


,x1,x2,y,y_pred,e,sample_weight,update_sample_weight
0,0.151174,0.354630,1,1,0,0.01,0.01
1,0.415353,0.984219,0,0,0,0.01,0.01
2,0.791493,0.903714,0,0,0,0.01,0.01
3,0.223697,0.077215,1,1,0,0.01,0.01
4,0.426906,0.121843,1,1,0,0.01,0.01
...,...,...,...,...,...,...,...
95,0.358278,0.046354,0,1,1,0.01,0.01
96,0.263334,0.842166,0,0,0,0.01,0.01
97,0.521146,0.931668,1,0,1,0.01,0.01
98,0.655226,0.428205,1,1,0,0.01,0.01


In [3423]:
total_sample_weight = sum(df['update_sample_weight'])
norm = round(df['update_sample_weight'] / total_sample_weight, 2)
df = df.assign(norm=norm)
df

,x1,x2,y,y_pred,e,sample_weight,update_sample_weight,norm
0,0.151174,0.354630,1,1,0,0.01,0.01,0.01
1,0.415353,0.984219,0,0,0,0.01,0.01,0.01
2,0.791493,0.903714,0,0,0,0.01,0.01,0.01
3,0.223697,0.077215,1,1,0,0.01,0.01,0.01
4,0.426906,0.121843,1,1,0,0.01,0.01,0.01
...,...,...,...,...,...,...,...,...
95,0.358278,0.046354,0,1,1,0.01,0.01,0.01
96,0.263334,0.842166,0,0,0,0.01,0.01,0.01
97,0.521146,0.931668,1,0,1,0.01,0.01,0.01
98,0.655226,0.428205,1,1,0,0.01,0.01,0.01


In [3424]:
df['range_upper'] = np.cumsum(df['norm'])
df['range_lower'] = df['range_upper'] - df['norm']
df

,x1,x2,y,y_pred,e,sample_weight,update_sample_weight,norm,range_upper,range_lower
0,0.151174,0.354630,1,1,0,0.01,0.01,0.01,0.01,0.00
1,0.415353,0.984219,0,0,0,0.01,0.01,0.01,0.02,0.01
2,0.791493,0.903714,0,0,0,0.01,0.01,0.01,0.03,0.02
3,0.223697,0.077215,1,1,0,0.01,0.01,0.01,0.04,0.03
4,0.426906,0.121843,1,1,0,0.01,0.01,0.01,0.05,0.04
...,...,...,...,...,...,...,...,...,...,...
95,0.358278,0.046354,0,1,1,0.01,0.01,0.01,0.96,0.95
96,0.263334,0.842166,0,0,0,0.01,0.01,0.01,0.97,0.96
97,0.521146,0.931668,1,0,1,0.01,0.01,0.01,0.98,0.97
98,0.655226,0.428205,1,1,0,0.01,0.01,0.01,0.99,0.98


In [3425]:
def create_new_dataset_kedua(df):
    indikasi = []
    for i in range(len(df)):
        a = np.random.uniform(0, 1)
        for index, row in df.iterrows():
            if a >= row['range_lower'] and a <= row['range_upper']:
                indikasi.append(index)
    return indikasi

indikasi = create_new_dataset_kedua(df)
indikasi



[63,
 57,
 59,
 17,
 86,
 52,
 8,
 69,
 89,
 5,
 86,
 29,
 35,
 87,
 78,
 37,
 49,
 78,
 95,
 31,
 70,
 37,
 64,
 90,
 69,
 27,
 53,
 13,
 42,
 61,
 26,
 52,
 1,
 95,
 64,
 9,
 48,
 27,
 55,
 65,
 43,
 93,
 77,
 84,
 69,
 63,
 76,
 32,
 42,
 61,
 67,
 80,
 89,
 35,
 21,
 76,
 5,
 33,
 0,
 40,
 33,
 56,
 63,
 41,
 75,
 23,
 19,
 26,
 35,
 98,
 55,
 42,
 21,
 67,
 4,
 53,
 13,
 46,
 53,
 58,
 41,
 8,
 39,
 0,
 89,
 94,
 86,
 72,
 34,
 48,
 89,
 61,
 88,
 54,
 61,
 35,
 47,
 62,
 59,
 7]

In [3426]:
dataset_kedua = []
dataset_kedua = df.loc[indikasi, ['x1', 'x2', 'y']]
dataset_kedua

,x1,x2,y
63,0.236652,0.948683,0
57,0.639319,0.993601,0
59,0.545204,0.498220,0
17,0.874585,0.996701,1
86,0.274492,0.771723,1
...,...,...,...
35,0.776465,0.535684,1
47,0.389021,0.601659,0
62,0.365003,0.685028,1
59,0.545204,0.498220,0


In [3427]:
knn2 = DecisionTreeClassifier(max_depth=1)
knn2 = knn2.fit(dataset_kedua[['x1', 'x2']], dataset_kedua['y'])
y_pred = knn2.predict(dataset_kedua[['x1', 'x2']])
dataset_kedua = dataset_kedua.assign(y_pred=y_pred)
dataset_kedua

,x1,x2,y,y_pred
63,0.236652,0.948683,0,0
57,0.639319,0.993601,0,0
59,0.545204,0.498220,0,0
17,0.874585,0.996701,1,1
86,0.274492,0.771723,1,0
...,...,...,...,...
35,0.776465,0.535684,1,1
47,0.389021,0.601659,0,0
62,0.365003,0.685028,1,0
59,0.545204,0.498220,0,0


In [3428]:
e = dataset_kedua['y'] != dataset_kedua['y_pred']
e = e.astype(int)
dataset_kedua = dataset_kedua.assign(e=e)
dataset_kedua

,x1,x2,y,y_pred,e
63,0.236652,0.948683,0,0,0
57,0.639319,0.993601,0,0,0
59,0.545204,0.498220,0,0,0
17,0.874585,0.996701,1,1,0
86,0.274492,0.771723,1,0,1
...,...,...,...,...,...
35,0.776465,0.535684,1,1,0
47,0.389021,0.601659,0,0,0
62,0.365003,0.685028,1,0,1
59,0.545204,0.498220,0,0,0


In [3429]:
sample_weight = 1/len(dataset_kedua)
dataset_kedua = dataset_kedua.assign(sample_weight=sample_weight)
dataset_kedua

,x1,x2,y,y_pred,e,sample_weight
63,0.236652,0.948683,0,0,0,0.01
57,0.639319,0.993601,0,0,0,0.01
59,0.545204,0.498220,0,0,0,0.01
17,0.874585,0.996701,1,1,0,0.01
86,0.274492,0.771723,1,0,1,0.01
...,...,...,...,...,...,...
35,0.776465,0.535684,1,1,0,0.01
47,0.389021,0.601659,0,0,0,0.01
62,0.365003,0.685028,1,0,1,0.01
59,0.545204,0.498220,0,0,0,0.01


In [3430]:
error_rate = sum(dataset_kedua['sample_weight'] * (dataset_kedua['y'] != dataset_kedua['y_pred']))
lamda_rate = round(1/2 * np.log((1-error_rate)/error_rate),2)
alpha_2 = lamda_rate
display(dataset_kedua)
print('error_rate: ', error_rate)
print('lamda_rate: ', lamda_rate)

,x1,x2,y,y_pred,e,sample_weight
63,0.236652,0.948683,0,0,0,0.01
57,0.639319,0.993601,0,0,0,0.01
59,0.545204,0.498220,0,0,0,0.01
17,0.874585,0.996701,1,1,0,0.01
86,0.274492,0.771723,1,0,1,0.01
...,...,...,...,...,...,...
35,0.776465,0.535684,1,1,0,0.01
47,0.389021,0.601659,0,0,0,0.01
62,0.365003,0.685028,1,0,1,0.01
59,0.545204,0.498220,0,0,0,0.01


error_rate:  0.3900000000000002
lamda_rate:  0.22


In [3431]:
update_sample_weight = []
for i in range(len(dataset_kedua)):
    if dataset_kedua['e'].iloc[i] == 1:
        update_sample_weight.append(round(dataset_kedua['sample_weight'].iloc[i] * np.exp(lamda_rate), 2))
    else:
        update_sample_weight.append(round(dataset_kedua['sample_weight'].iloc[i] * np.exp(-lamda_rate), 2))
dataset_kedua = dataset_kedua.assign(update_sample_weight=update_sample_weight)
dataset_kedua

,x1,x2,y,y_pred,e,sample_weight,update_sample_weight
63,0.236652,0.948683,0,0,0,0.01,0.01
57,0.639319,0.993601,0,0,0,0.01,0.01
59,0.545204,0.498220,0,0,0,0.01,0.01
17,0.874585,0.996701,1,1,0,0.01,0.01
86,0.274492,0.771723,1,0,1,0.01,0.01
...,...,...,...,...,...,...,...
35,0.776465,0.535684,1,1,0,0.01,0.01
47,0.389021,0.601659,0,0,0,0.01,0.01
62,0.365003,0.685028,1,0,1,0.01,0.01
59,0.545204,0.498220,0,0,0,0.01,0.01


In [3432]:
total_sample_weight = sum(dataset_kedua['update_sample_weight'])
norm = round(dataset_kedua['update_sample_weight'] / total_sample_weight, 2)
dataset_kedua = dataset_kedua.assign(norm=norm)
dataset_kedua

,x1,x2,y,y_pred,e,sample_weight,update_sample_weight,norm
63,0.236652,0.948683,0,0,0,0.01,0.01,0.01
57,0.639319,0.993601,0,0,0,0.01,0.01,0.01
59,0.545204,0.498220,0,0,0,0.01,0.01,0.01
17,0.874585,0.996701,1,1,0,0.01,0.01,0.01
86,0.274492,0.771723,1,0,1,0.01,0.01,0.01
...,...,...,...,...,...,...,...,...
35,0.776465,0.535684,1,1,0,0.01,0.01,0.01
47,0.389021,0.601659,0,0,0,0.01,0.01,0.01
62,0.365003,0.685028,1,0,1,0.01,0.01,0.01
59,0.545204,0.498220,0,0,0,0.01,0.01,0.01


In [3433]:
dataset_kedua['range_upper'] = np.cumsum(dataset_kedua['norm'])
dataset_kedua['range_lower'] = dataset_kedua['range_upper'] - dataset_kedua['norm']
dataset_kedua

,x1,x2,y,y_pred,e,sample_weight,update_sample_weight,norm,range_upper,range_lower
63,0.236652,0.948683,0,0,0,0.01,0.01,0.01,0.01,0.00
57,0.639319,0.993601,0,0,0,0.01,0.01,0.01,0.02,0.01
59,0.545204,0.498220,0,0,0,0.01,0.01,0.01,0.03,0.02
17,0.874585,0.996701,1,1,0,0.01,0.01,0.01,0.04,0.03
86,0.274492,0.771723,1,0,1,0.01,0.01,0.01,0.05,0.04
...,...,...,...,...,...,...,...,...,...,...
35,0.776465,0.535684,1,1,0,0.01,0.01,0.01,0.96,0.95
47,0.389021,0.601659,0,0,0,0.01,0.01,0.01,0.97,0.96
62,0.365003,0.685028,1,0,1,0.01,0.01,0.01,0.98,0.97
59,0.545204,0.498220,0,0,0,0.01,0.01,0.01,0.99,0.98


In [3434]:
def create_new_dataset_ketiga(dataset_kedua):
    indikasi = []
    for i in range(len(dataset_kedua)):
        a = np.random.uniform(0, 1)
        for index, row in dataset_kedua.iterrows():
            if a >= row['range_lower'] and a <= row['range_upper']:
                indikasi.append(index)
    return indikasi

indikasi_kedua = create_new_dataset_ketiga(dataset_kedua)
indikasi_kedua

[80,
 89,
 27,
 8,
 39,
 48,
 95,
 89,
 94,
 64,
 33,
 63,
 56,
 31,
 35,
 42,
 35,
 35,
 76,
 89,
 89,
 37,
 55,
 86,
 76,
 49,
 37,
 94,
 0,
 9,
 33,
 98,
 69,
 53,
 53,
 8,
 31,
 41,
 5,
 70,
 37,
 0,
 39,
 75,
 34,
 55,
 41,
 46,
 86,
 61,
 64,
 63,
 80,
 94,
 37,
 64,
 53,
 95,
 41,
 37,
 0,
 69,
 80,
 19,
 95,
 69,
 89,
 63,
 61,
 69,
 53,
 78,
 29,
 95,
 0,
 94,
 31,
 42,
 26,
 46,
 53,
 33,
 39,
 33,
 17,
 37,
 95,
 78,
 32,
 9,
 61,
 53,
 64,
 65,
 27,
 80,
 64,
 86,
 41,
 65]

In [3435]:
dataset_ketiga = []
print(indikasi_kedua)
dataset_ketiga = dataset_kedua.loc[indikasi_kedua, ['x1', 'x2', 'y']]
dataset_ketiga

[80, 89, 27, 8, 39, 48, 95, 89, 94, 64, 33, 63, 56, 31, 35, 42, 35, 35, 76, 89, 89, 37, 55, 86, 76, 49, 37, 94, 0, 9, 33, 98, 69, 53, 53, 8, 31, 41, 5, 70, 37, 0, 39, 75, 34, 55, 41, 46, 86, 61, 64, 63, 80, 94, 37, 64, 53, 95, 41, 37, 0, 69, 80, 19, 95, 69, 89, 63, 61, 69, 53, 78, 29, 95, 0, 94, 31, 42, 26, 46, 53, 33, 39, 33, 17, 37, 95, 78, 32, 9, 61, 53, 64, 65, 27, 80, 64, 86, 41, 65]


,x1,x2,y
80,0.996983,0.623483,0
89,0.075311,0.814911,0
89,0.075311,0.814911,0
89,0.075311,0.814911,0
89,0.075311,0.814911,0
...,...,...,...
86,0.274492,0.771723,1
86,0.274492,0.771723,1
41,0.103490,0.610932,0
41,0.103490,0.610932,0


In [3436]:
knn3 = DecisionTreeClassifier(max_depth=1)
knn3 = knn3.fit(dataset_ketiga[['x1', 'x2']], dataset_ketiga['y'])
y_pred = knn3.predict(dataset_ketiga[['x1', 'x2']])
dataset_ketiga = dataset_ketiga.assign(y_pred=y_pred)
dataset_ketiga

,x1,x2,y,y_pred
80,0.996983,0.623483,0,0
89,0.075311,0.814911,0,0
89,0.075311,0.814911,0,0
89,0.075311,0.814911,0,0
89,0.075311,0.814911,0,0
...,...,...,...,...
86,0.274492,0.771723,1,0
86,0.274492,0.771723,1,0
41,0.103490,0.610932,0,0
41,0.103490,0.610932,0,0


In [3437]:
e = dataset_ketiga['y'] != dataset_ketiga['y_pred']
e = e.astype(int)
dataset_ketiga = dataset_ketiga.assign(e=e)
dataset_ketiga

,x1,x2,y,y_pred,e
80,0.996983,0.623483,0,0,0
89,0.075311,0.814911,0,0,0
89,0.075311,0.814911,0,0,0
89,0.075311,0.814911,0,0,0
89,0.075311,0.814911,0,0,0
...,...,...,...,...,...
86,0.274492,0.771723,1,0,1
86,0.274492,0.771723,1,0,1
41,0.103490,0.610932,0,0,0
41,0.103490,0.610932,0,0,0


In [3438]:
sample_weight = 1/len(dataset_ketiga)
dataset_ketiga = dataset_ketiga.assign(sample_weight=sample_weight)
dataset_ketiga

,x1,x2,y,y_pred,e,sample_weight
80,0.996983,0.623483,0,0,0,0.004762
89,0.075311,0.814911,0,0,0,0.004762
89,0.075311,0.814911,0,0,0,0.004762
89,0.075311,0.814911,0,0,0,0.004762
89,0.075311,0.814911,0,0,0,0.004762
...,...,...,...,...,...,...
86,0.274492,0.771723,1,0,1,0.004762
86,0.274492,0.771723,1,0,1,0.004762
41,0.103490,0.610932,0,0,0,0.004762
41,0.103490,0.610932,0,0,0,0.004762


In [3439]:
error_rate = sum(dataset_ketiga['sample_weight'] * (dataset_ketiga['y'] != dataset_ketiga['y_pred']))
print(error_rate)
lamda_rate = round(1/2 * np.log((1-error_rate)/error_rate),2)
alpha_3 = lamda_rate
display(dataset_ketiga)
print('error_rate: ', error_rate)
print('lamda_rate: ', lamda_rate)

0.39523809523809494


,x1,x2,y,y_pred,e,sample_weight
80,0.996983,0.623483,0,0,0,0.004762
89,0.075311,0.814911,0,0,0,0.004762
89,0.075311,0.814911,0,0,0,0.004762
89,0.075311,0.814911,0,0,0,0.004762
89,0.075311,0.814911,0,0,0,0.004762
...,...,...,...,...,...,...
86,0.274492,0.771723,1,0,1,0.004762
86,0.274492,0.771723,1,0,1,0.004762
41,0.103490,0.610932,0,0,0,0.004762
41,0.103490,0.610932,0,0,0,0.004762


error_rate:  0.39523809523809494
lamda_rate:  0.21


In [3440]:
update_sample_weight = []
for i in range(len(dataset_ketiga)):
    if dataset_ketiga['e'].iloc[i] == 1:
        update_sample_weight.append(round(dataset_ketiga['sample_weight'].iloc[i] * np.exp(lamda_rate), 2))
    else:
        update_sample_weight.append(round(dataset_ketiga['sample_weight'].iloc[i] * np.exp(-lamda_rate), 2))
dataset_ketiga = dataset_ketiga.assign(update_sample_weight=update_sample_weight)
dataset_ketiga

,x1,x2,y,y_pred,e,sample_weight,update_sample_weight
80,0.996983,0.623483,0,0,0,0.004762,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00
...,...,...,...,...,...,...,...
86,0.274492,0.771723,1,0,1,0.004762,0.01
86,0.274492,0.771723,1,0,1,0.004762,0.01
41,0.103490,0.610932,0,0,0,0.004762,0.00
41,0.103490,0.610932,0,0,0,0.004762,0.00


In [3441]:
total_sample_weight = sum(dataset_ketiga['update_sample_weight'])
norm = round(dataset_ketiga['update_sample_weight'] / total_sample_weight, 2)
dataset_ketiga = dataset_ketiga.assign(norm=norm)
dataset_ketiga

,x1,x2,y,y_pred,e,sample_weight,update_sample_weight,norm
80,0.996983,0.623483,0,0,0,0.004762,0.00,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00,0.00
...,...,...,...,...,...,...,...,...
86,0.274492,0.771723,1,0,1,0.004762,0.01,0.01
86,0.274492,0.771723,1,0,1,0.004762,0.01,0.01
41,0.103490,0.610932,0,0,0,0.004762,0.00,0.00
41,0.103490,0.610932,0,0,0,0.004762,0.00,0.00


In [3442]:
dataset_ketiga['range_upper'] = np.cumsum(dataset_ketiga['norm'])
dataset_ketiga['range_lower'] = dataset_ketiga['range_upper'] - dataset_ketiga['norm']
dataset_ketiga

,x1,x2,y,y_pred,e,sample_weight,update_sample_weight,norm,range_upper,range_lower
80,0.996983,0.623483,0,0,0,0.004762,0.00,0.00,0.00,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00,0.00,0.00,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00,0.00,0.00,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00,0.00,0.00,0.00
89,0.075311,0.814911,0,0,0,0.004762,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
86,0.274492,0.771723,1,0,1,0.004762,0.01,0.01,0.81,0.80
86,0.274492,0.771723,1,0,1,0.004762,0.01,0.01,0.82,0.81
41,0.103490,0.610932,0,0,0,0.004762,0.00,0.00,0.82,0.82
41,0.103490,0.610932,0,0,0,0.004762,0.00,0.00,0.82,0.82


In [3443]:
prediksi_1 = knn1.predict(x)
prediksi_2 = knn2.predict(x)
prediksi_3 = knn3.predict(x)

print(prediksi_1)
print(prediksi_2)
print(prediksi_3)

[1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 0 0 1 1]
[0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0
 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1
 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [3444]:
print(alpha_1)
print(alpha_2)
total_alpha_x_pred = (alpha_1 * prediksi_1) + (alpha_2 * prediksi_2) + (alpha_3 * prediksi_3)
print(total_alpha_x_pred)

0.16
0.22
[0.16 0.   0.22 0.16 0.16 0.16 0.38 0.16 0.16 0.16 0.16 0.16 0.16 0.38
 0.38 0.16 0.38 0.22 0.16 0.16 0.   0.16 0.   0.38 0.16 0.16 0.16 0.16
 0.16 0.16 0.   0.38 0.38 0.16 0.16 0.38 0.16 0.16 0.38 0.16 0.16 0.16
 0.16 0.16 0.38 0.38 0.38 0.16 0.16 0.16 0.16 0.16 0.   0.38 0.16 0.38
 0.22 0.   0.16 0.16 0.38 0.16 0.16 0.   0.16 0.38 0.16 0.22 0.16 0.16
 0.16 0.   0.16 0.38 0.16 0.   0.16 0.16 0.16 0.38 0.38 0.38 0.16 0.16
 0.16 0.38 0.16 0.16 0.16 0.   0.16 0.16 0.   0.22 0.16 0.16 0.   0.
 0.16 0.38]


In [3445]:
print('prediksi: ', np.sign(total_alpha_x_pred))
accuracy = accuracy_score(y, np.sign(total_alpha_x_pred))
print('accuracy: ', accuracy)

prediksi:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 1. 1.]
accuracy:  0.59
